In [ ]:
import os,sys,inspect
from IPython.display import display, Markdown, Latex, Javascript, clear_output
import emoji
import time
import ipywidgets as widgets
import ipython_blocking
import numpy as np
import time
from tqdm import tqdm_notebook
import functools
import configparser
import webbrowser
import shutil

parentdir = os.path.abspath('../../../pymodules')

sys.path.insert(0,parentdir)
#from connect_device import connect
from connect_device_v2.vizualizate import vizulizated_finds_devices, advanced_search_device,DeviceBundle
from connect_device_v2.save_file import SaveFilesButton
from connect_device_v2.connect import connect_to_device
from connect_device_v2.find_device import FindDevice
from connect_device_v2.auxiliary_commands import check_error,disable_all_correctors,enable_ifbf_for_all_segment
import pandas as pd
from datetime import datetime
button = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )
testname="Тест неравномерности АЧХ"
display(Markdown('### 💠 {}'.format(testname)))



In [ ]:
from connect_device_v2.find_device import FindDevice
box = FindDevice()
display(box)

In [ ]:
button = box.select_device_continue_button

In [ ]:
%blockrun button

In [ ]:
button1 = box.select_config_continue_button

In [ ]:
%blockrun button1

In [ ]:
inst = box.selected_device

In [ ]:
inst.write("SYST:PRES")

In [ ]:
date=datetime.now()
date=date.strftime('%Y-%m-%d %H:%M:%S')
device = inst.query("*IDN?")
device = device.replace(' ','')
device = device.split(',')
device_info = {'Время запуска':date,
              'device':device[0],
              'model':device[1],
              'serial':device[2],
              'version':device[3]}
#inst2 = ng.instance()
#путь к файлу с сегментами
currentDir=%pwd
#resultPath= currentDir +'\\frequency_plan.seg'
portCount=int(inst.query("SERV:PORT:COUN?"))
traces=["S","R","T"]
traces_count = portCount* len(traces)

In [ ]:
def parse_config(config: dict):
    factor_value = {
        'k': 10 ** 3,
        'm': 10 ** 6,
        'g': 10 ** 9
    }
    new_config = config.copy()
    for i in config['RANGE']:
        value = config['RANGE'][i]
        try:
            numeric, factor = value.split(' ')
            new_value = float(numeric) * factor_value[factor.lower()]
            # print(new_value)
            new_config['RANGE'][i] = str(new_value)
        except:
            pass
    return new_config

In [ ]:
result_config =  box.config


In [ ]:
def generate_admitance_table(config):
    admittance_table = {}
    range_count = len(config['ADMITANCE'])/2
    range_num = 1
    markers = []
    for key in config['ADMITANCE']:

        if markers == (key.split('_')[1::]):
            pass      

        else:
            markers = (key.split('_')[1::])

            try:

                next_marker = config['ADMITANCE'][f'TR_{markers[0]}_{int(markers[1])}']

                admittance_table[range_num]={
                'start':config['RANGE'][f'marker_{markers[0]}'],
                'stop':config['RANGE'][f'marker_{markers[1]}'],
                'admitance_s':config['ADMITANCE'][key],
                'admitance_tr':next_marker,          
                }
                range_num+=1
            except:
                pass
    return admittance_table

In [ ]:
admittance_table = generate_admitance_table(result_config)

range_count = len(result_config['RANGE'])-1



In [ ]:
admittance_table2 = generate_admitance_table(parse_config(dict(result_config)))

In [ ]:
markers_table = {}
for key in result_config['RANGE']:
    if 'marker' in key:
        marker_number = key.split('_',3)[1::]
        #print(marker_number)
        markers_table[marker_number[0]]=result_config['RANGE'][key]

In [ ]:
# def check_error():
#     responce = inst.query('SYST:ERR?')    
#     if "No error" not in responce:        
#         print(responce)
#         #pass
#     else:        
#         pass
#         #print('succes')

In [ ]:
filter_IF=result_config['DEVICE']['filter_IF']
athenuator=result_config['DEVICE']['athenuator']
#Сброс по умолчанию и остановка сканирования
#inst.write("*RST")
inst.write("SYST:PRES")
inst.write("INIT:CONT 0")
inst.write("SERVice:SWEep:FREQuency:FACTory")
enable_ifbf_for_all_segment(inst)
# Фильтр ПЧ 1 КГц 
inst.write(f"SENS:BAND {filter_IF}")


#Частотный план
#inst.write("SENS:SWE:TYPE SEGMent")

#inst.write("MMEMory:LOAD:SEGM {}".format(resultPath))



# атенюатор SERVice:RFCTL:POWer:STATe 1
inst.write(f"SERVice:RFCTL:POWer:STATe {athenuator}")
inst.write(f"SERVice:RFCTL:POWer:ATT 10")
inst.write(f"SERV:RFCTL:POW:DAC 6554")

#Отключение всех коррекций
disable_all_correctors(inst)
print()
#inst.write('CALC:PAR:COUN {}'.format(range_count))


######################################################

In [ ]:
trace_param = []
for port in range(1,portCount+1):
    for trace in traces:    
        if trace is not 'S':
            trace_param.append(f'{trace}{port}')
        else : 
            trace_param.append(f'{trace}{port}{port}')
        

In [ ]:
def setup_markers(markers_table):
    #print(f"CALC:MARK:COUN {len(markers_table)+1}")
    inst.write(f"CALC:MARK:COUN {len(markers_table)+1}")  
    #check_error()
    #inst.write("CALC:MARK:COUP 0") # выключить свзяность
    #print(f"CALC:MARK{len(markers_table)+1} 1")
    #inst.write(f"CALC:MARK{len(markers_table)+1} 1")
    #check_error()
    for key in markers_table:        
        inst.write("CALC:PAR1:SEL")
        #check_error()
        
        marker_value_start = markers_table[key]['start']
        marker_value_stop = markers_table[key]['stop']        
        
        #print(f"CALC:MARK{key}:X {marker_value_start}")
        inst.write(f"CALC:MARK{key}:X {marker_value_start}")
        #check_error()        
        inst.write(f"CALC:MARK{key+1}:X {marker_value_stop}")
        #check_error()
                
                

In [ ]:
def setup_trace(trace):
    #print(f"CALC:PAR:DEF {trace}")
    inst.write(f"CALC:PAR:DEF {trace}")    
    #check_error()
    #print('CALC:MST 1')
    inst.write('CALC:MST 1')    
    #check_error()
    #print('CALC:MST:DOM 1')
    inst.write('CALC:MST:DOM 1')
    #check_error()
    
    

In [ ]:
def run_scan():
    inst.write("INIT")
    while(True):
        status=inst.query("TRIG:STAT?")
        if (status!="HOLD"):
            time.sleep(1)
            
        else :
            #print("scan {} successful".format(traceName))
            break
    

In [ ]:

def get_freq_responce(markers_table):
    result_table = []
    result_string = ()
    #marker_value1 = table_markers[i-1].value
    #marker_value2 = table_markers[i].value
    #inst.write("CALC:PAR{}:SEL".format(i))
    
    #inst.write('CALC:MST:DOM 1')
    i=0
    for key in markers_table:
        
        marker_num = key
        
        inst.write(f'CALC:MST:DOM:STOP {marker_num+1}')        
        inst.write(f'CALC:MST:DOM:START {marker_num}')
        
        inst.write(f'CALC:MST:DOM 1')
        result = inst.query_ascii_values('CALC:MST:DATA? ')[2]        
        result = round(result,4)        
        
        trace_name =inst.query("CALC:PAR:DEF?")
        
        if 'S' in trace_name:            
            class_trace = 'S'    
            result_string = (result,
                             0,
                             markers_table[key]['admitance_s'],                            )
            
            result_table.append(list(result_string))
        else:
            class_trace = 'TR'
            result_string = (result,
                             0,
                             markers_table[key]['admitance_tr'],
                            )
            result_table.append(list(result_string))
    return((class_trace,trace_name,result_table))
    #return((trace_name,result_table))
    

In [ ]:
result_table = []
setup_markers(admittance_table2)
pbar = tqdm_notebook(total=portCount*len(traces))
for trace in trace_param:    
    setup_trace(trace)  
    #time.sleep(0.5)
    run_scan()    
    a=get_freq_responce(admittance_table)
    result_table.append(a)
    pbar.update(1)


In [ ]:
table=sorted(result_table,key=lambda result_table: result_table[0])


In [ ]:
def get_max(table):
    #print(table)
    range_num=0
    
    result=[]
    name = ''
    while (range_num<range_count):        
        temp=[]
        values=[]
        for i in table:
            name = i[1][0]            
            temp.append(i[2][range_num][0])       
        
       
        values.append(max(temp))
        result.append(values)
        
        range_num+=1
    return (name,result)


In [ ]:
temp_s=[]
temp_tr = []
for i in table:
    if 'S' in i[0]:
        temp_s.append(i)
    else:
        temp_tr.append(i)
temp_s1=get_max(temp_s)
temp_tr1=get_max(temp_tr)


In [ ]:

for i in table:
    
    if 'S' in i[0]:
        range_number=0
        
        while(range_number<range_count):
            if (i[2][range_number][0]<temp_s1[1][range_number][0]):
                #print(i[1][range_number][1])
                i[2][range_number][1]=0
            else:
                #print(i[1])
                #print(range_number)
                #print('else' ,temp_s1[1][range_number][0])
                i[2][range_number][1]=temp_s1[1][range_number][0]
            range_number+=1
    else:
        range_number=0
       
        while(range_number<range_count):
            if (i[2][range_number][0]<temp_tr1[1][range_number][0]):
                #print(i[1][range_number][1])
                i[2][range_number][1]=0
            else:
                #print(i[1])
                #print(range_number)
                #print('else' ,temp_s1[1][range_number][0])
                i[2][range_number][1]=temp_tr1[1][range_number][0]
            range_number+=1
        
        
            

In [ ]:
def table_generator(table):
    html_table='<table border="1">\
    <tr>\
        <td>Диапазон</td>'
    td_background="bgcolor=\"#ffdd00\""    
    for key in admittance_table:
        start=admittance_table[key]['start']
        stop=admittance_table[key]['stop']
        
        html_table += f"<td colspan=3>{start} - {stop}</td>" 
    html_table += "</tr>"
        #######
    html_table += "<tr><td>Трассы</td>"
    for i in range (int(range_count)):
        html_table += "<td>Нер-ть,дБ</td> \
                        <td>Макс.дБ</td> \
                        <td>Допуск,дБ</td> "
    html_table += "</tr>"
    for i in table:
        html_table += "<tr>"
        html_table += f'<td>{i[1]}</td>'
        for j in i[2::]:
            for k in j:
                html_table += f'<td>{k[0]}</td>'  
                #print(k)
                if k[1]>float(k[2]):                    
                    html_table +=f'<td {td_background}>{k[1]}</td>'
                    html_table +=f'<td {td_background}>{k[2]}</td>'
                    continue
                if (k[1]!=0):
                    html_table +=f'<td >{k[1]}</td>'
                    html_table +=f'<td >{k[2]}</td>'
                    continue
                else:
                    html_table +=f'<td>-</td>'
                    html_table +=f'<td>{k[2]}</td>'
                    
            
        
   
            
        html_table +='</tr>'
    html_table += "</table>"
    return html_table

In [ ]:
def html_header(data):
    html_table='<table border="1"><tr>'
    for key in data:
        html_table+=f'<td>{key}</td>'
    html_table+=f'</tr>'
    html_table+=f'<tr>'
    for key in data:
        html_table+=f'<td>{data[key]}</td>'
    html_table+=f'</tr>'    
    return html_table

In [ ]:
from IPython.core.display import display, HTML
html_table=table_generator(table)
header = html_header(device_info)


display(HTML(header),HTML(html_table))

In [ ]:
#with open() as f:
#    f.write(header+html_table)

In [ ]:
file_data = header+html_table
my_button = SaveFilesButton(file_data)
my_button # This will display the button in the context of Jupyter Notebook